<a href="https://colab.research.google.com/github/MengOonLee/WebScrapy/blob/master/Groceries/Lotus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
pip install --no-cache-dir -qU scrapy selenium

In [43]:
%%writefile Lotus.py
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import scrapy
from scrapy.crawler import CrawlerProcess
import logging
logging.disable("WARNING")

class LotusSpider(scrapy.Spider):
    name = 'Lotus'

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        options = webdriver.chrome.options.Options()
        options.add_argument("--headless")
        options.add_argument("--no-sandbox")
        options.add_argument("--enable-javascript")
        options.add_argument("--enable-cookies")
        options.add_argument("--disable-notifications")
        options.add_argument("--disable-web-security")
        options.add_argument("--incognito")
        self.driver = webdriver.Chrome(options=options)

    def start_requests(self):
        urls = [
            "https://www.lotuss.com.my/en/product/aus-lamb-shoulder-slice-5319-74539329"
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse_items)

    def parse_categories(self, response):
        self.driver.get(response.url)

        elem_present = ""
        while not elem_present:
            try:
                elem_present = self.driver.find_element(By.XPATH,
                    "//div[@class='product-grid-item']"
                )
            except:
                continue

        selector = scrapy.Selector(text=self.driver.page_source)
        category = selector.css("h1#category-title::text").get()
        print(category)
        category_links = selector.css("div.carousel a")
        if len(category_links)!=0:
            yield from response.follow_all(category_links,
                callback=self.parse_categories)
        else:
            last_height = self.driver.execute_script(
                "return document.body.scrollHeight"
            )

            while True:
                self.driver.execute_script(
                    "window.scrollTo(0, document.body.scrollHeight)"
                )
                time.sleep(3)
                new_height = self.driver.execute_script(
                    "return document.body.scrollHeight"
                )
                if new_height==last_height:
                    break
                last_height = new_height

            selector = scrapy.Selector(text=self.driver.page_source)
            item_links = selector.css("div#product-list a")

            yield from response.follow_all(item_links,
                callback=self.parse_items)

    def parse_items(self, response):
        self.driver.get(response.url)

        elem_present = ""
        while not elem_present:
            try:
                elem_present = self.driver.find_element(By.XPATH, """
                    //img[@id='current-product-image']
                """)
            except:
                continue

        selector = scrapy.Selector(text=self.driver.page_source)
        name = selector.css("div.MuiBox-root h1::text").get()
        price = selector.css("div.MuiBox-root ::text")[3:6].getall()
        price = "".join(price)
        info = selector.css("div.MuiBox-root::text")[2].get()
        print(name, price, info)

process = CrawlerProcess()
process.crawl(LotusSpider)
process.start()

Overwriting Lotus.py


In [44]:
%%bash
python Lotus.py

LOTUS'S FRESH AUS LAMB SHOULDER SLICE (5319) RM28.97/Kg Elevate your culinary creations with Lotus's Fresh AUS Lamb Shoulder Slice. This premium cut of lamb, sourced from Australia, offers tender and succulent slices perfect for grilling, roasting, or simmering in your favorite dishes. Enjoy the quality and flavor of Australian lamb for a delightful dining experience.


/usr/local/lib/python3.10/dist-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting for information on how to handle this deprecation.
  return cls(crawler)
